In [43]:
import numpy as np

def bfgs(f, x0, grad_f, tol=1e-6, maxiter=100):
    n = x0.shape[0]
    I = np.eye(n)
    x = x0
    H = I
    grad = grad_f(x)
    iter = 0
   
    while np.linalg.norm(grad) > tol and iter < maxiter:
        p = -np.dot(H, grad)
        alpha = 1.0
        while f(x + alpha*p) > f(x) + alpha*0.1*np.dot(grad, p):
            alpha *= 0.5
        s = alpha*p
        x_new = x + s
        grad_new = grad_f(x_new)
        y = grad_new - grad
        rho = 1.0 / np.dot(y, s)
        
        H = (I - rho*np.outer(s, y.T)) @ H @ (I - rho*np.outer(y, s.T)) + rho*np.outer(s, s.T)
        x = x_new
        
        grad = grad_new
        iter += 1
    return [x,iter]



In [48]:

def dfp(f, x0, grad_f, tol=1e-6, maxiter=100):
    n = x0.shape[0]
    I = np.eye(n)
    x = x0
    H = I
    grad = grad_f(x)
    iter = 0
    while np.linalg.norm(grad) > tol and iter < maxiter:
        p = -np.dot(H, grad)
        alpha = 1.0
        while f(x + alpha*p) > f(x) + alpha*0.1*np.dot(grad, p):
            alpha *= 0.5
        s = alpha*p
        x_new = x + s
        grad_new = grad_f(x_new)
        y = grad_new - grad
        rho = 1.0 / np.dot(y, s)
        A = I - rho*np.outer(s, y)
        B = I - rho*np.outer(y, s)
        H = np.dot(A, np.dot(H, B)) + rho*np.outer(s, s)
        x = x_new
        grad = grad_new
        iter += 1
    return [x, iter]

import numpy as np

def sr1_root(f, grad_f, x0, tol=1e-6, maxiter=1000):
    n = x0.shape[0]
    H = np.eye(n)
    x = x0
    grad = grad_f(x)
    iter = 0
    while np.linalg.norm(grad) > tol and iter < maxiter:
        p = -np.dot(H, grad)
        alpha = 1.0
        while f(x + alpha*p) > f(x) + alpha*0.1*np.dot(grad, p):
            alpha *= 0.5
        s = alpha*p
        x_new = x + s
        grad_new = grad_f(x_new)
        y = grad_new - grad
        # Compute the update s that results in the change in gradient y
        if np.abs(np.dot(s, y)) >= tol*np.linalg.norm(s)*np.linalg.norm(y):
            # SR1 update if the denominator is non-zero
            
            H += np.outer((s - np.dot(H, y)), (s - np.dot(H, y))) / np.dot(s - np.dot(H, y), y)
        # If the denominator is close to zero, skip the update
        x = x_new
        grad = grad_new
        iter += 1
    return [x, iter]


In [50]:
import numpy as np
from scipy.optimize import minimize as opt

# Define the Rosenbrock function and its gradient
import numpy as np

def rosenbrock(x, a=1, b=100):
    
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2

def rosenbrock_gradient(x, a=1, b=100):
   
    grad = np.zeros_like(x)
    grad[0] = 2*(x[0] - a) - 4*b*x[0]*(x[1] - x[0]**2)
    grad[1] = 2*b*(x[1] - x[0]**2)
    return grad

# Set the initial guess
x0 = np.array([-1.2, 1.0])
x1 = np.array([0.5,0.5])
# Use the BFGS algorithm to minimize the Rosenbrock function
result = minimize(rosenbrock, x0, method='BFGS', jac=rosenbrock_grad)
result1 = bfgs(rosenbrock, x0, rosenbrock_grad)
result2 = dfp(rosenbrock, x0, rosenbrock_grad, )
result3= sr1_root(rosenbrock, rosenbrock_grad, x0)
# Print the optimized solution and function value
print("Optimized solution:", result.x)
print("Optimized function value:", result.fun)
print(result1)
print(result2)
print(result3)

Optimized solution: [0.99999997 0.99999995]
Optimized function value: 2.5353055595473786e-15
[array([1.        , 0.99999999]), 34]
[array([1.        , 0.99999999]), 34]
[array([nan, nan]), 5]


<ipython-input-48-7779bab2f3ee>:46: RuntimeWarning: divide by zero encountered in true_divide
  H += np.outer((s - np.dot(H, y)), (s - np.dot(H, y))) / np.dot(s - np.dot(H, y), y)
